# Demo (structuring)

Injection of first order logic rules into a neural network for iris classification task via structuring.

Some imports.

In [37]:
from psyki import KnowledgeModule
from psyki.fol import Parser
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from test import get_mlp
from test import get_rules
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.python.keras.layers import Dense, Concatenate
from tensorflow.keras.optimizers import Adam


Loading iris dataset and separation into train and test set.

In [38]:
x, y = load_iris(return_X_y=True, as_frame=True)
encoder = OneHotEncoder(sparse=False)
encoder.fit_transform([y])
dataset = x.join(y)

train, test = train_test_split(dataset, test_size=0.5, random_state=0)
train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
3,4.6,3.1,1.5,0.2,0
149,5.9,3.0,5.1,1.8,2
98,5.1,2.5,3.0,1.1,1
6,4.6,3.4,1.4,0.3,0
68,6.2,2.2,4.5,1.5,1
...,...,...,...,...,...
9,4.9,3.1,1.5,0.1,0
103,6.3,2.9,5.6,1.8,2
67,5.8,2.7,4.1,1.0,1
117,7.7,3.8,6.7,2.2,2


Import FOL rules:

- PL <= 2.28 <- X = setosa
- PL > 2.28 ^ PW > 1.64 <- X = virginica
- PL > 2.28 ^ PW <= 1.64 <- X = versicolor

In [39]:
features_mapping = {
    'SL': 0,
    'SW': 1,
    'PL': 2,
    'PW': 3,
}
class_mapping = {
    'setosa': tf.constant([1, 0, 0], dtype=tf.float32),
    'virginica': tf.constant([0, 1, 0], dtype=tf.float32),
    'versicolor': tf.constant([0, 0, 1], dtype=tf.float32)
}

parser = Parser.default_parser()
iris_rules = get_rules('iris')

Injection of fuzzy logic function derived from FOL rules into a neural network.

In [40]:
input_features = Input((4,), name='Input')
network = get_mlp(input=input_features, output=3, layers=3, neurons=16, activation_function='relu',
                  last_activation_function='softmax')
kns = KnowledgeModule.modules(iris_rules, parser, input_features, features_mapping)
model = Model(input_features, network).layers[-1].output
output = Dense(10, activation='softmax')((Concatenate(axis=1)([model] + kns)))
model = Model(input_features, output)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 4)]          0                                            
__________________________________________________________________________________________________
lambda_44 (Lambda)              (None, 1)            0           Input[0][0]                      
__________________________________________________________________________________________________
dense_106 (Dense)               (None, 1)            5           Input[0][0]                      
__________________________________________________________________________________________________
lambda_36 (Lambda)              (None, 1)            0           Input[0][0]                      
____________________________________________________________________________________________

Training.

In [41]:
train_x, train_y = train.iloc[:,:-1], train.iloc[:,-1]
model.fit(train_x, train_y, verbose=1, batch_size=5, epochs=50)

Epoch 1/50
15/15 [==============================] - 0s 468us/step - loss: 2.2717 - accuracy: 0.0667
Epoch 2/50
15/15 [==============================] - 0s 612us/step - loss: 2.2004 - accuracy: 0.0800
Epoch 3/50
15/15 [==============================] - 0s 496us/step - loss: 2.1278 - accuracy: 0.0800
Epoch 4/50
15/15 [==============================] - 0s 489us/step - loss: 2.0484 - accuracy: 0.0800
Epoch 5/50
15/15 [==============================] - 0s 500us/step - loss: 1.9530 - accuracy: 0.0800
Epoch 6/50
15/15 [==============================] - 0s 511us/step - loss: 1.8587 - accuracy: 0.1

Evaluation.

In [42]:
test_x, test_y = test.iloc[:,:-1], test.iloc[:,-1]
model.evaluate(test_x, test_y)

3/3 [==============================] - 0s 714us/step - loss: 0.5706 - accuracy: 0.9867


[0.5706334710121155, 0.9866666793823242]

### Demo ends here
If you are reading it from `https://anonymous.4open.science/` there is a chance that the demo is duplicated.
Just ignore the following text.